In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
EXECUTE_SCRIPT = True

In [ ]:
ENV = 'D'
DATABASE_NAME = f'D_WAGERING_WHS_DB'

In [ ]:
DB_ROLE_TYPE_READ = "RO"
DB_ROLE_TYPE_WRITE = "RW"

In [ ]:
SNOWFLAKE_SELECT_ITEMS = ["TABLES", "VIEWS", "EXTERNAL TABLES", "DYNAMIC TABLES", "ICEBERG TABLES", "MATERIALIZED VIEWS", "STREAMS"]
SNOWFLAKE_REFERENCES_ITEMS = ["TABLES", "VIEWS", "EXTERNAL TABLES", "ICEBERG TABLES", "MATERIALIZED VIEWS"]

SNOWFLAKE_USAGE_ITEMS = ["FUNCTIONS", "PROCEDURES", "FILE FORMATS", "STAGES", "SEQUENCES"]
SNOWFLAKE_MONITOR_ITEMS = ["TASKS", "ALERTS", "DYNAMIC TABLES"]
SNOWFLAKE_READ_ITEMS = ["STAGES"]

SNOWFLAKE_ETL_ITEMS = ["TABLES", "ICEBERG TABLES"]
SNOWFLAKE_WRITE_ITEMS = ["STAGES"]
SNOWFLAKE_OPERATE_ITEMS = ["DYNAMIC TABLES", "ALERTS", "TASKS"]
SNOWFLAKE_ETL_MONITOR_ITEMS = ["PIPES"]


In [ ]:
NEW_LINE = "\n"

In [ ]:
USE ROLE ACCOUNTADMIN;

In [ ]:
rslt = session.sql(f"SHOW SCHEMAS IN DATABASE {DATABASE_NAME}").collect()
filtered_schemas = [row for row in rslt if row.name not in ('INFORMATION_SCHEMA', 'PUBLIC')]


In [ ]:
for row in filtered_schemas:
    schema_name = row.name
    schema_role_name_read = f"SC_{schema_name}_{DB_ROLE_TYPE_READ}"

    sql_script = f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"
    
    for type in SNOWFLAKE_SELECT_ITEMS:
        sql_script += f"REVOKE SELECT ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} FROM DATABASE ROLE {schema_role_name_read};{NEW_LINE}"

    for type in SNOWFLAKE_REFERENCES_ITEMS:
        sql_script += f"REVOKE REFERENCES ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} FROM DATABASE ROLE {schema_role_name_read};{NEW_LINE}"

    for type in SNOWFLAKE_USAGE_ITEMS:
        sql_script += f"REVOKE USAGE ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} FROM DATABASE ROLE {schema_role_name_read};{NEW_LINE}"

    for type in SNOWFLAKE_MONITOR_ITEMS:
        sql_script += f"REVOKE MONITOR ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} FROM DATABASE ROLE {schema_role_name_read};{NEW_LINE}"

    for type in SNOWFLAKE_READ_ITEMS:
        sql_script += f"REVOKE READ ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} FROM DATABASE ROLE {schema_role_name_read};{NEW_LINE}"

    sql_script += f"{NEW_LINE}{NEW_LINE}"
    print (f"--Starting generating script for schema {DATABASE_NAME}.{schema_name}");
    print (sql_script);

    
    if EXECUTE_SCRIPT:
        
        print (f"--Execute script for schema {DATABASE_NAME}.{schema_name}");
        session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
        session.sql(sql_script).collect()
    

In [ ]:
for row in filtered_schemas:
    schema_name = row.name
    schema_role_name_write = f"SC_{schema_name}_{DB_ROLE_TYPE_WRITE}"

    sql_script = f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"

    for type in SNOWFLAKE_ETL_ITEMS:
        sql_script += f"REVOKE INSERT, UPDATE, DELETE, TRUNCATE ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} FROM DATABASE ROLE {schema_role_name_write};{NEW_LINE}"

    for type in SNOWFLAKE_WRITE_ITEMS:
        sql_script += f"REVOKE READ, WRITE ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} FROM DATABASE ROLE {schema_role_name_write};{NEW_LINE}"

    for type in SNOWFLAKE_OPERATE_ITEMS:
        sql_script += f"REVOKE OPERATE ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} FROM DATABASE ROLE {schema_role_name_write};{NEW_LINE}"

    for type in SNOWFLAKE_ETL_MONITOR_ITEMS:
        sql_script += f"REVOKE MONITOR ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} FROM DATABASE ROLE {schema_role_name_write};{NEW_LINE}"

    sql_script += f"{NEW_LINE}{NEW_LINE}"
    print (f"--Starting generating script for schema {schema_name}");
    print (sql_script);

    if EXECUTE_SCRIPT:
        print (f"--Execute script for schema {DATABASE_NAME}.{schema_name}");
        session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
        session.sql(sql_script).collect()
    

In [ ]:
for row in filtered_schemas:
    schema_name = f"{row.name}"
        
    sql_script = f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"
    sql_script += f"ALTER SCHEMA {DATABASE_NAME}.{row.name} DISABLE MANAGED ACCESS;"
    print (sql_script);

    if EXECUTE_SCRIPT:
        session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
        session.sql(sql_script).collect()
        